In [7]:
import time
import numpy as np
import pandas as pd
import nltk
from sklearn.datasets import load_digits
from sklearn import preprocessing

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
#print('data :',data)
#print(type(data))

In [3]:
df = pd.read_csv('Emotions_final.csv')

In [4]:
#stopwords = nltk.corpus.stopwords.words("english")
#stopwords.append('feel')
#stopwords.append('feeling')
corpus = df['Text']
targets = df['Emotion']

le = preprocessing.LabelEncoder()
le.fit(targets)
labels = le.transform(targets)

print(corpus.shape)
print(labels.shape)


(21459,)
(21459,)


In [5]:
#vectorisation corpus
vectorizer = CountVectorizer()
data = vectorizer.fit_transform(corpus)

In [6]:
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(with_mean=False), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean")
    ]

    # Show the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}")
    print(formatter_result.format(*results))



Plantages à répétition sur la suite!!!!!!!!!!!!!!!!!!! =>Ok fit_transform voir PCA

In [8]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD

n_digits= 6
#pca.components_= 5

print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

kmeans = KMeans(init="k-means++", n_clusters=n_digits, n_init=1)
bench_k_means(kmeans=kmeans, name="k-means++", data=data, labels=labels)


kmeans = KMeans(init="random", n_clusters=n_digits, n_init=1, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=data, labels=labels)

svd =TruncatedSVD(n_components=6)          ######### error: float() argument must be a string or a number, not 'TruncatedSVD'
svd_data=svd.fit_transform(data)
#pca = PCA(n_components=n_digits).fit(data)
kmeans = KMeans(init=svd.components_, n_clusters=n_digits, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=svd_data, labels=labels)

print(82 * '_')


__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette


TypeError: 'module' object is not callable